In [1]:
print('Hello World')

Hello World


In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
import re
import time

In [ ]:
# 크롬옵션
chrome_options = Options()
# 꺼짐 방지
chrome_options.add_argument('--disable-gpu') 
# 창 크기에 따라 XPATH가 달라져 크기 고정
chrome_options.add_argument("--start-maximized")
# 알림표시 제거
chrome_options.add_argument("--disable-notifications")
# 옵션 적용
driver = webdriver.Chrome(options=chrome_options)

wan_ins_url = '''https://insight.wanted.co.kr/'''
driver.get(wan_ins_url)

In [ ]:
# 기업 검색창 클릭
driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/nav/div/div[2]/a/div[2]').click()
# 기업 입력
driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/form/div/div/input').send_keys(f'{기업명}')
time.sleep(0.5)
# 엔터
driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/form/div/div/input').send_keys(Keys.ENTER)

employee = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[1]/div[2]/div[2]/div[1]/span').text
sales_money = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[1]/div[3]/div[2]/div[1]/span').text
sector = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[1]/dd').text
corp_year = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[2]/dd').text
corp_url = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[8]/dd').text

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
import re
import time

last_les = pd.DataFrame(
    columns = ['기업명', '사원수', '설립연도', '매출액', '기업 홈페이지 URL ']
    )

for corp_name in df.기업명:
    # 크롬옵션
    chrome_options = Options()
    # 꺼짐 방지
    chrome_options.add_argument('--disable-gpu') 
    # 창 크기에 따라 XPATH가 달라져 크기 고정
    chrome_options.add_argument("--start-maximized")
    # 알림표시 제거
    chrome_options.add_argument("--disable-notifications")
    # 옵션 적용
    driver = webdriver.Chrome(options=chrome_options)
    wan_ins_url = '''https://insight.wanted.co.kr/'''
    driver.get(wan_ins_url)

    # 기업 검색창 클릭
    driver.find_element(
        By.XPATH,
        '//*[@id="__next"]/div[1]/nav/div/div[2]/a/div[2]'
        ).click()
    # 기업 입력
    driver.find_element(
        By.XPATH, 
        '//*[@id="__next"]/div[2]/div/div/form/div/div/input'
        ).send_keys(f'{corp_name}')
    time.sleep(0.6)
    # 엔터
    driver.find_element(
        By.XPATH, 
        '//*[@id="__next"]/div[2]/div/div/form/div/div/input'
        ).send_keys(Keys.ENTER)
    
    try:
        #Crawling-data
        employee = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[1]/div[2]/div[2]/div[1]/span').text
        sales_money = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[1]/div[3]/div[2]/div[1]/span').text
        sector = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[1]/dd').text
        corp_year = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[2]/dd').text
        corp_url = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[8]/dd').text
        #요소 못찾을시 결측치 처리    
    except NoSuchElementException:
        None

    # Data concat
    last_temp = pd.DataFrame(
        [corp_name, employee, corp_year, sales_money, corp_url],
        index = ['기업명', '사원수', '설립연도', '매출액', '기업 홈페이지 URL ']
    ).T
    last_les = pd.concat([last_les, last_temp])

